# Backtesting Trading Strategies in Python

---

## 🧠 Introduction:
Welcome! This notebook is designed to help you learn how to backtest trading strategies using Python. You'll learn how to:

- Download financial data
- Build and test a basic trading strategy (Moving Average Crossover)
- Backtest the strategy's performance
- Visualize it using line charts and candlesticks
- Add basic performance metrics (like Sharpe Ratio)
- Organize your project for future expansion

Whether you're a beginner or building toward more advanced quant skills, this walkthrough will help you gain a practical foundation in backtesting.


---

## 📁 Suggested Project Folder Structure

To keep your backtesting projects clean and organized, use the following folder layout:

```
backtesting_project/
│
├── data/                   ← Historical market data (CSV or downloaded)
├── strategies/             ← Your trading strategy functions
│   └── moving_average.py   ← A basic moving average crossover strategy
├── notebooks/              ← Jupyter notebooks like this one
│   └── backtest_demo.ipynb ← This learning notebook
├── utils/                  ← Plotting, performance metrics, helper tools
│   └── plot_tools.py       ← Candlestick and returns plotting
├── main.py                 ← Optional: run backtest from script (non-notebook)
├── requirements.txt        ← Package list
└── README.md               ← Overview of your project (optional)


In [ ]:
# !pip install yfinance pandas matplotlib numpy mplfinance

In [4]:
%pip install yfinance mplfinance

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplfinance as mpf


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
def download_data(ticker="AAPL", start="2020-01-01", end="2024-01-01"):
    data = yf.download(ticker, start=start, end=end)
    data = data[['Open', 'High', 'Low', 'Close']]
    data.dropna(inplace=True)
    return data

# Example usage:
data = download_data()
data.head()


In [ ]:
def generate_signals(data, short_window=20, long_window=50):
    data['Short_MA'] = data['Close'].rolling(window=short_window).mean()
    data['Long_MA'] = data['Close'].rolling(window=long_window).mean()
    data['Signal'] = 0
    data['Signal'][short_window:] = np.where(
        data['Short_MA'][short_window:] > data['Long_MA'][short_window:], 1, 0
    )
    data['Position'] = data['Signal'].diff()
    return data

strategy_data = generate_signals(data)
strategy_data[['Close', 'Short_MA', 'Long_MA', 'Signal', 'Position']].tail()


In [ ]:
def backtest_strategy(data):
    data['Daily_Return'] = data['Close'].pct_change()
    data['Strategy_Return'] = data['Daily_Return'] * data['Signal'].shift(1)
    cumulative_strategy = (1 + data['Strategy_Return']).cumprod()
    cumulative_market = (1 + data['Daily_Return']).cumprod()
    return cumulative_strategy, cumulative_market

cumulative_strategy, cumulative_market = backtest_strategy(strategy_data)


In [ ]:
def plot_performance(strategy, market):
    plt.figure(figsize=(12,6))
    plt.plot(strategy, label="Strategy")
    plt.plot(market, label="Market (Buy & Hold)")
    plt.title("Strategy vs Market Performance")
    plt.xlabel("Date")
    plt.ylabel("Cumulative Returns")
    plt.legend()
    plt.grid(True)
    plt.show()

plot_performance(cumulative_strategy, cumulative_market)


In [ ]:
def plot_candlesticks(data):
    mpf.plot(data, type='candle', mav=(20, 50), volume=False, style='yahoo')

plot_candlesticks(strategy_data)


In [ ]:
def calculate_sharpe_ratio(data, risk_free_rate=0.01):
    excess_return = data['Strategy_Return'] - (risk_free_rate / 252)
    sharpe_ratio = np.sqrt(252) * excess_return.mean() / excess_return.std()
    return sharpe_ratio

sharpe = calculate_sharpe_ratio(strategy_data)
print(f"Sharpe Ratio: {sharpe:.2f}")


---

## 🧩 How to Add Your Own Strategies
You can add and test custom trading strategies using two main methods:

### 🔸 Option 1: Inline in the Notebook
Great for fast testing. Create a new function inside the notebook:
```python
def generate_rsi_signals(data):
    # Your RSI logic here
    return data

strategy_data = generate_rsi_signals(data)
```

### 🔸 Option 2: As a Separate Python File (Best Practice)
Create a new file like `strategies/rsi_strategy.py`:
```python
# strategies/rsi_strategy.py

def generate_rsi_signals(data):
    # Your RSI logic here
    return data
```
Then use it in your notebook:
```python
from strategies.rsi_strategy import generate_rsi_signals
strategy_data = generate_rsi_signals(data)
```

➡️ Just make sure your strategy function returns the modified `data` DataFrame with at least these columns:
- `'Signal'`: 1 for long, 0 for flat
- `'Position'`: the difference in signals (used to track trades)


---

## ✅ Next Steps
Now that you've completed a full backtest pipeline, here are options to explore:

🔹 **Try new strategies**:
- RSI-based strategy (momentum)
- Bollinger Bands (volatility-based)
- MACD or Mean Reversion

🔹 **Improve performance analytics**:
- Add drawdown analysis
- Calculate max drawdown, win rate, etc.

🔹 **Refactor and modularize**:
- Move strategies to `strategies/`
- Move plots to `utils/`
- Create reusable scripts

🔹 **Advanced tools**:
- Try `backtrader`, `bt`, or `vectorbt` for advanced simulations

You now have a solid foundation to build and backtest your own algorithmic trading strategies in Python.

Happy coding and trading! 🚀
